In [59]:
import pandas as pd
import numpy as numpy
import nltk
nltk.download(['punkt','averaged_perceptron_tagger','wordnet'])
from nltk.corpus import stopwords
import string
import contractions
import langdetect
from langdetect import detect 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\seung\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\seung\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\seung\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [19]:
# Open file
with open('main/data/youtube_comments.csv',encoding="utf8") as file:
    df = pd.read_csv(file)
file.close()

In [20]:
# check head
df.head()

,url,link_title,channel,no_of_views,time_uploaded,comment,author,comment_posted,no_of_replies,upvotes,downvotes
0,https://www.youtube.com/watch?v=UXSqcCYMFE4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","I swear, whoever is playing Plague Inc needs t...",Leo Hurtt,NaN,NaN,486,NaN
1,https://www.youtube.com/watch?v=UXSqcCYMFE4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",I am getting real tired of being part of majo...,Sergio Villasenor,NaN,NaN,426,NaN
2,https://www.youtube.com/watch?v=UXSqcCYMFE4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",This channel's COVID reports are a model of ho...,Eyerleth,NaN,NaN,797,NaN
3,https://www.youtube.com/watch?v=UXSqcCYMFE4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","To people who understand science, the developm...",gowzahr,NaN,NaN,105,NaN
4,https://www.youtube.com/watch?v=UXSqcCYMFE4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",So with these new variants this whole thing fe...,Itsonlyafleshwound,NaN,NaN,205,NaN


In [21]:
# drop unnecessary columns
df.drop(['comment_posted','no_of_replies','downvotes','url','author'],axis=1, inplace=True)

In [22]:
# Check for missing values
for col in df.columns:
    print(col, df[col].isnull().sum())

link_title 0
channel 0
no_of_views 0
time_uploaded 0
comment 0
upvotes 145


In [23]:
# fill null values for upvotes ( = 0 likes) as 0
df["upvotes"].fillna(0, inplace=True)

In [24]:
# Check for missing values
for col in df.columns:
    print(col, df[col].isnull().sum())

link_title 0
channel 0
no_of_views 0
time_uploaded 0
comment 0
upvotes 0


In [25]:
df.head(5)

,link_title,channel,no_of_views,time_uploaded,comment,upvotes
0,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","I swear, whoever is playing Plague Inc needs t...",486
1,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",I am getting real tired of being part of majo...,426
2,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",This channel's COVID reports are a model of ho...,797
3,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","To people who understand science, the developm...",105
4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",So with these new variants this whole thing fe...,205


In [26]:
### expanding contractions = don't --> do not ###

# add a column with contractions removed
df['no_contract'] = df['comment'].apply(lambda x: [contractions.fix(word) for word in x.split() ])
df.loc[10]

link_title       We Know More About Those COVID-19 Variants. It...
channel                                                    SciShow
no_of_views                                           85,566 views
time_uploaded                                         Jan 30, 2021
comment          If there's anything good from this, it's that ...
upvotes                                                          9
no_contract      [If, there is, anything, good, from, this,, it...
Name: 10, dtype: object

In [27]:
# though sentences are tokenized, the contractions are tokenized into one token
# i.e. We've --> ["we have"] instead of ["we", "have"]
# we'll join the tokens back into singular strings so we can tokenize them later

# map(str,l) takes the iterable l (all values in df['no_contract']) and stringifies it w/ str
df['comments_str'] = [' '.join(map(str, l)) for l in df['no_contract']]
df.head()

,link_title,channel,no_of_views,time_uploaded,comment,upvotes,no_contract,comments_str
0,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","I swear, whoever is playing Plague Inc needs t...",486,"[I, swear,, whoever, is, playing, Plague, Inc,...","I swear, whoever is playing Plague Inc needs t..."
1,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",I am getting real tired of being part of majo...,426,"[I, am, getting, real, tired, of, being, part,...",I am getting real tired of being part of major...
2,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",This channel's COVID reports are a model of ho...,797,"[This, channel's, COVID, reports, are, a, mode...",This channel's COVID reports are a model of ho...
3,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","To people who understand science, the developm...",105,"[To, people, who, understand, science,, the, d...","To people who understand science, the developm..."
4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",So with these new variants this whole thing fe...,205,"[So, with, these, new, variants, this, whole, ...",So with these new variants this whole thing fe...


In [54]:
#tokenize the comments using NLTK
df['tokenized'] = df['comments_str'].apply(word_tokenize)
df.head()

# convert characters to lowercase
df['lower'] = df['tokenized'].apply(lambda x: [word.lower() for word in x])

# remove punctuations
punct = string.punctuation
# make a list only if token is not a punctuation
df['no_punct'] = df['lower'].apply(lambda x: [word for word in x if word not in punct])

# remove stopwords
stop_words = stopwords.words('english')
df['no_stopwords'] = df['no_punct'].apply(lambda x: [word for word in x if word not in stop_words])

df.head()

,link_title,channel,no_of_views,time_uploaded,comment,upvotes,no_contract,comments_str,tokenized,lower,no_punct,no_stopwords
0,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","I swear, whoever is playing Plague Inc needs t...",486,"[I, swear,, whoever, is, playing, Plague, Inc,...","I swear, whoever is playing Plague Inc needs t...","[I, swear, ,, whoever, is, playing, Plague, In...","[i, swear, ,, whoever, is, playing, plague, in...","[i, swear, whoever, is, playing, plague, inc, ...","[swear, whoever, playing, plague, inc, needs, ..."
1,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",I am getting real tired of being part of majo...,426,"[I, am, getting, real, tired, of, being, part,...",I am getting real tired of being part of major...,"[I, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[getting, real, tired, part, major, historical..."
2,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",This channel's COVID reports are a model of ho...,797,"[This, channel's, COVID, reports, are, a, mode...",This channel's COVID reports are a model of ho...,"[This, channel, 's, COVID, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[channel, 's, covid, reports, model, science, ..."
3,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","To people who understand science, the developm...",105,"[To, people, who, understand, science,, the, d...","To people who understand science, the developm...","[To, people, who, understand, science, ,, the,...","[to, people, who, understand, science, ,, the,...","[to, people, who, understand, science, the, de...","[people, understand, science, development, vac..."
4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",So with these new variants this whole thing fe...,205,"[So, with, these, new, variants, this, whole, ...",So with these new variants this whole thing fe...,"[So, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[new, variants, whole, thing, feels, like, fou..."


In [57]:
# Normalization - lemmatizing
df['pos_tags'] = df['no_stopwords'].apply(nltk.tag.pos_tag)
df.head()

,link_title,channel,no_of_views,time_uploaded,comment,upvotes,no_contract,comments_str,tokenized,lower,no_punct,no_stopwords,pos_tags
0,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","I swear, whoever is playing Plague Inc needs t...",486,"[I, swear,, whoever, is, playing, Plague, Inc,...","I swear, whoever is playing Plague Inc needs t...","[I, swear, ,, whoever, is, playing, Plague, In...","[i, swear, ,, whoever, is, playing, plague, in...","[i, swear, whoever, is, playing, plague, inc, ...","[swear, whoever, playing, plague, inc, needs, ...","[(swear, JJ), (whoever, WP), (playing, VBG), (..."
1,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",I am getting real tired of being part of majo...,426,"[I, am, getting, real, tired, of, being, part,...",I am getting real tired of being part of major...,"[I, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[getting, real, tired, part, major, historical...","[(getting, VBG), (real, JJ), (tired, JJ), (par..."
2,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",This channel's COVID reports are a model of ho...,797,"[This, channel's, COVID, reports, are, a, mode...",This channel's COVID reports are a model of ho...,"[This, channel, 's, COVID, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[channel, 's, covid, reports, model, science, ...","[(channel, NN), ('s, POS), (covid, NN), (repor..."
3,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","To people who understand science, the developm...",105,"[To, people, who, understand, science,, the, d...","To people who understand science, the developm...","[To, people, who, understand, science, ,, the,...","[to, people, who, understand, science, ,, the,...","[to, people, who, understand, science, the, de...","[people, understand, science, development, vac...","[(people, NNS), (understand, VBP), (science, N..."
4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",So with these new variants this whole thing fe...,205,"[So, with, these, new, variants, this, whole, ...",So with these new variants this whole thing fe...,"[So, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[new, variants, whole, thing, feels, like, fou...","[(new, JJ), (variants, NNS), (whole, JJ), (thi..."


In [61]:
# we convert the speech tags into the appropriate wordnet format
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('r'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# replace the pos tag with wordnet tags
df['wordnet_pos'] = df['pos_tags'].apply(lambda x : [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [63]:
df.head()

,link_title,channel,no_of_views,time_uploaded,comment,upvotes,no_contract,comments_str,tokenized,lower,no_punct,no_stopwords,pos_tags,wordnet_pos
0,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","I swear, whoever is playing Plague Inc needs t...",486,"[I, swear,, whoever, is, playing, Plague, Inc,...","I swear, whoever is playing Plague Inc needs t...","[I, swear, ,, whoever, is, playing, Plague, In...","[i, swear, ,, whoever, is, playing, plague, in...","[i, swear, whoever, is, playing, plague, inc, ...","[swear, whoever, playing, plague, inc, needs, ...","[(swear, JJ), (whoever, WP), (playing, VBG), (...","[(swear, a), (whoever, n), (playing, v), (plag..."
1,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",I am getting real tired of being part of majo...,426,"[I, am, getting, real, tired, of, being, part,...",I am getting real tired of being part of major...,"[I, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[getting, real, tired, part, major, historical...","[(getting, VBG), (real, JJ), (tired, JJ), (par...","[(getting, v), (real, a), (tired, a), (part, n..."
2,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",This channel's COVID reports are a model of ho...,797,"[This, channel's, COVID, reports, are, a, mode...",This channel's COVID reports are a model of ho...,"[This, channel, 's, COVID, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[channel, 's, covid, reports, model, science, ...","[(channel, NN), ('s, POS), (covid, NN), (repor...","[(channel, n), ('s, n), (covid, n), (reports, ..."
3,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","To people who understand science, the developm...",105,"[To, people, who, understand, science,, the, d...","To people who understand science, the developm...","[To, people, who, understand, science, ,, the,...","[to, people, who, understand, science, ,, the,...","[to, people, who, understand, science, the, de...","[people, understand, science, development, vac...","[(people, NNS), (understand, VBP), (science, N...","[(people, n), (understand, v), (science, n), (..."
4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",So with these new variants this whole thing fe...,205,"[So, with, these, new, variants, this, whole, ...",So with these new variants this whole thing fe...,"[So, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[new, variants, whole, thing, feels, like, fou...","[(new, JJ), (variants, NNS), (whole, JJ), (thi...","[(new, a), (variants, n), (whole, a), (thing, ..."


In [69]:
# lemmatize comments
lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['wordnet_pos'].apply(lambda x: [lemmatizer.lemmatize(word,tag) for (word,tag) in x])

df.head()

,link_title,channel,no_of_views,time_uploaded,comment,upvotes,no_contract,comments_str,tokenized,lower,no_punct,no_stopwords,pos_tags,wordnet_pos,lemmatized
0,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","I swear, whoever is playing Plague Inc needs t...",486,"[I, swear,, whoever, is, playing, Plague, Inc,...","I swear, whoever is playing Plague Inc needs t...","[I, swear, ,, whoever, is, playing, Plague, In...","[i, swear, ,, whoever, is, playing, plague, in...","[i, swear, whoever, is, playing, plague, inc, ...","[swear, whoever, playing, plague, inc, needs, ...","[(swear, JJ), (whoever, WP), (playing, VBG), (...","[(swear, a), (whoever, n), (playing, v), (plag...","[swear, whoever, play, plague, inc, need, stop..."
1,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",I am getting real tired of being part of majo...,426,"[I, am, getting, real, tired, of, being, part,...",I am getting real tired of being part of major...,"[I, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[i, am, getting, real, tired, of, being, part,...","[getting, real, tired, part, major, historical...","[(getting, VBG), (real, JJ), (tired, JJ), (par...","[(getting, v), (real, a), (tired, a), (part, n...","[get, real, tired, part, major, historical, ev..."
2,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",This channel's COVID reports are a model of ho...,797,"[This, channel's, COVID, reports, are, a, mode...",This channel's COVID reports are a model of ho...,"[This, channel, 's, COVID, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[this, channel, 's, covid, reports, are, a, mo...","[channel, 's, covid, reports, model, science, ...","[(channel, NN), ('s, POS), (covid, NN), (repor...","[(channel, n), ('s, n), (covid, n), (reports, ...","[channel, 's, covid, report, model, science, r..."
3,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021","To people who understand science, the developm...",105,"[To, people, who, understand, science,, the, d...","To people who understand science, the developm...","[To, people, who, understand, science, ,, the,...","[to, people, who, understand, science, ,, the,...","[to, people, who, understand, science, the, de...","[people, understand, science, development, vac...","[(people, NNS), (understand, VBP), (science, N...","[(people, n), (understand, v), (science, n), (...","[people, understand, science, development, vac..."
4,We Know More About Those COVID-19 Variants. It...,SciShow,"85,566 views","Jan 30, 2021",So with these new variants this whole thing fe...,205,"[So, with, these, new, variants, this, whole, ...",So with these new variants this whole thing fe...,"[So, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[so, with, these, new, variants, this, whole, ...","[new, variants, whole, thing, feels, like, fou...","[(new, JJ), (variants, NNS), (whole, JJ), (thi...","[(new, a), (variants, n), (whole, a), (thing, ...","[new, variant, whole, thing, feel, like, fough..."


In [70]:
# finally, save cleaned data into csv file
df.to_csv('youtube_comments_clean.csv')